# Libraries

In [1]:
import json
import pandas as pd

from nltk.tokenize import RegexpTokenizer

import gc
from thefuzz import fuzz
import numpy as np
from tqdm.auto import tqdm

from catboost import CatBoostClassifier, Pool

import warnings

In [2]:
pd.set_option('display.max_columns', 256)
warnings.filterwarnings('ignore')

# Data

In [3]:
train_pairs = pd.read_parquet('./datasets/train_pairs_w_target.parquet')
train_pairs['target'] = train_pairs['target'].astype(int)
rtrain_pairs = train_pairs.copy()
rtrain_pairs.variantid1, rtrain_pairs.variantid2 = rtrain_pairs.variantid2, rtrain_pairs.variantid1
train_pairs

,target,variantid1,variantid2
0,0,51197862,51198054
1,1,53062686,536165289
2,1,53602615,587809782
3,1,53888651,89598677
4,0,56930698,551526166
...,...,...,...
306535,0,817327230,822083612
306536,0,817560551,818069912
306537,0,817854719,817857267
306538,0,820036017,820037019


In [4]:
test_pairs = pd.read_parquet('./datasets/test_pairs_wo_target.parquet')
rtest_pairs = test_pairs.copy()
rtest_pairs.variantid1, rtest_pairs.variantid2 = rtest_pairs.variantid2, rtest_pairs.variantid1
test_pairs

,variantid1,variantid2
0,52076340,290590137
1,64525522,204128919
2,77243372,479860557
3,86065820,540678372
4,91566575,258840506
...,...,...
18079,666998614,667074522
18080,670036240,670048449
18081,670284509,684323809
18082,692172005,704805270


In [6]:
train_data = pd.read_parquet('./datasets/train_data.parquet').set_index('variantid')
train_data['characteristic_attributes_mapping'] = train_data['characteristic_attributes_mapping'].fillna('{}').apply(lambda x: json.loads(x))
train_data['categories'] = train_data['categories'].apply(lambda x: json.loads(x))
train_data['main_pic_embeddings_resnet_v1'] = train_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
train_data = pd.concat([train_data, pd.read_parquet('./datasets/name_labse_embs_train.parquet').set_index('variantid')], axis=1)
train_data = pd.concat([train_data, pd.read_pickle('./datasets/neuro_embeds_train.pickle')], axis=1)
train_data

,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping,name_labse_768,neuro_768
variantid,,,,,,,,,
51195767,"Удлинитель Партнер-Электро ПВС 2х0,75 ГОСТ,6A,...","{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",[оранжевый],None,"[0.04603629, 0.18839523, -0.09973055, -0.66368...","[-0.47045058, 0.67237014, 0.48984158, -0.54485...","{'Номинальный ток, А': ['10'], 'Цвет товара': ...","[-0.033874325, 0.03722446, 0.0029757991, 0.068...","[0.05308385, -0.035102457, 0.035711396, -0.003..."
53565809,Магнитный кабель USB 2.0 A (m) - USB Type-C (m...,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",[красный],"[[0.26863545, -0.3130674, 0.29023397, 0.073978...","[1.1471839, -0.665361, 0.7745614, 0.26716197, ...","[-0.6575592, 0.6522429, 0.5426037, -0.54347897...",{'Конструктивные особенности': ['Магнитная кон...,"[0.015568526, -0.03899538, 0.064447366, 0.0383...","[0.019788187, 0.0360369, -0.05522197, 0.010044..."
56763357,"Набор микропрепаратов Konus 25: ""Клетки и ткан...","{'1': 'EPG', '2': 'Электроника', '3': 'Оптичес...",None,"[[0.66954195, 1.0643557, 0.78324044, -0.338267...","[-0.90570974, 1.0296293, 1.0769907, 0.27746, -...","[-0.7384308, 0.70784587, 0.3012653, -0.3583719...","{'Тип аксессуара': ['Набор микропрепаратов'], ...","[-0.033072222, -0.04237577, 0.020771954, 0.065...",NaN
56961772,"Мобильный телефон BQ 1848 Step, черный","{'1': 'EPG', '2': 'Электроника', '3': 'Смартфо...",[черный],"[[0.6580482, -0.35763323, -0.16939065, -0.4249...","[0.13133773, -0.5577079, 0.32498044, 0.1917174...","[-0.44812852, 0.5283565, 0.28981736, -0.506841...","{'Тип карты памяти': ['microSD'], 'Число SIM-к...","[0.014727573, -0.025661988, 0.023943473, -0.00...",NaN
61054740,"Штатив трипод Tripod 330A для фотоаппаратов, в...","{'1': 'EPG', '2': 'Электроника', '3': 'Штативы...",[черный],"[[-0.10406649, 0.080646515, -0.28668788, 0.739...","[0.21696381, 0.10989461, -0.08012986, 0.691861...","[-0.72692573, 0.75206333, 0.37740713, -0.52502...","{'Материал': ['Металл'], 'Количество секций, ш...","[0.043145332, -0.052424084, 0.017260496, 0.045...","[-0.0063637993, 0.040778656, -0.018901959, 0.0..."
...,...,...,...,...,...,...,...,...,...
820128810,"Комплект 2 шт, Чернила Cactus CS-EPT6733B пурп...","{'1': 'EPG', '2': 'Электроника', '3': 'Расходн...",[пурпурный],None,"[-1.4492652, -0.80129164, -0.12344764, 0.71945...","[-0.8253241, 0.6785133, 0.53978086, -0.4888316...","{'Тип': ['Чернила для принтера'], 'Бренд печат...","[-0.003678058, -0.031628493, 0.0065589263, 0.0...",NaN
821135769,"Защитное стекло закаленное Xiaomi Redmi 7, Y3 ...","{'1': 'EPG', '2': 'Электроника', '3': 'Защитны...",[черный],"[[0.09564891, 0.27437285, -0.19054827, -0.7992...","[0.012127608, -0.8534423, 0.5415518, -0.449125...","[-0.7413257, 0.46105132, 0.5639801, -0.5462132...","{'Вид стекла': ['3D'], 'Тип': ['Защитное стекл...","[-0.06858361, 0.027011767, -0.016400583, -0.02...","[0.070231415, -0.012321442, -0.06056241, -0.02..."
822095690,Системный блок ЮКОМС 9400-268 (AMD A6-9400 (3....,"{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",[черный],None,"[0.4248176, -0.15944786, -0.22844064, 0.427686...","[-0.49261805, 0.56726897, 0.7037877, -0.697246...","{'Общий объем HDD, ГБ': ['10000'], 'Видеокарта...","[-0.04474233, -0.034224413, 0.026076552, 0.026...",NaN


In [7]:
test_data = pd.read_parquet('./datasets/test_data.parquet').set_index('variantid')
test_data['characteristic_attributes_mapping'] = test_data['characteristic_attributes_mapping'].fillna('{}').apply(lambda x: json.loads(x))
test_data['categories'] = test_data['categories'].apply(lambda x: json.loads(x))
test_data['main_pic_embeddings_resnet_v1'] = test_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
test_data = pd.concat([test_data, pd.read_parquet('./datasets/name_labse_embs_test.parquet').set_index('variantid')], axis=1)
test_data = pd.concat([test_data, pd.read_pickle('./datasets/neuro_embeds_test.pickle')], axis=1)
test_data

,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping,name_labse_768,neuro_768
variantid,,,,,,,,,
51201254,Колодка TDM Electric четырехместная без заземл...,"{'1': 'EPG', '2': 'Электроника', '3': 'Сетевые...",[белый],"[[0.34383398, -0.2962618, 0.07987049, -0.08257...","[0.38310742, -0.7876679, 0.5018278, 0.20900711...","[-0.5060825, 0.5773388, 0.59435517, -0.4958292...","{'Страна-изготовитель': ['Китай'], 'Бренд': ['...","[-0.0058242553, 0.0010011946, 0.015051351, 0.0...","[-0.037606854, 0.08384453, -0.04124399, 0.0028..."
77151532,Клавиатура черная с черной рамкой для 25-011879,"{'1': 'EPG', '2': 'Электроника', '3': 'Запчаст...",[черный],None,"[0.50964713, 0.7958329, -1.4113188, 0.19993813...","[-0.43467724, 0.6614495, 0.48050267, -0.588880...","{'Страна-изготовитель': ['Китай'], 'Комплектац...","[0.0088402, -0.0050699823, 0.026550002, -0.015...","[-0.052735697, 0.00019977099, 0.015895747, -0...."
89664856,"15.6"" Игровой ноутбук Acer Predator Helios 300...","{'1': 'EPG', '2': 'Электроника', '3': 'Компьют...",[черный],"[[0.7804302, -0.245446, -0.67754817, -0.614691...","[0.9958085, -0.113175124, -0.7623152, -0.91648...","[-0.70010763, 0.48152006, 0.47597092, -0.51727...",{'Видеокарта': ['NVIDIA GeForce RTX 2070 (8 Гб...,"[-0.026623247, -0.018851712, 0.011397564, -0.0...","[-0.00044866692, -0.07461986, -0.017260032, -0..."
90701982,Портативная колонка Borofone BR7 Empyreal Spor...,"{'1': 'EPG', '2': 'Электроника', '3': 'Акустик...","[red, красный]","[[-0.24636984, -1.0719914, -0.49986655, 0.3423...","[-0.26596686, -1.143009, -0.5289628, 0.4285588...","[-0.73135185, -0.039796613, 0.38907066, -0.496...","{'Основной материал корпуса': ['Металл'], 'Мак...","[0.016539363, 0.03778109, 0.025718935, 0.08805...","[0.0173606, 0.039641324, -0.024564413, -0.0056..."
92484118,Аккумулятор для Meizu BA712 ( M6s ),"{'1': 'EPG', '2': 'Электроника', '3': 'Батарей...",None,None,"[0.42047608, 0.75828516, 0.5440093, -0.0068945...","[-0.600158, 0.13944691, 0.48706242, -0.5050975...","{'Рекомендовано для': ['Meizu'], 'Бренд': ['Me...","[-0.0024493372, 0.02346121, 0.068452105, 0.023...","[0.04657542, -0.027531484, -0.015035957, 0.059..."
...,...,...,...,...,...,...,...,...,...
702785891,Кабель USB - Lightning HOCO X21 PLUS (черно-бе...,"{'1': 'EPG', '2': 'Электроника', '3': 'Кабели ...",[черный],"[[1.1820095, -0.16312826, 1.4916217, 0.0288323...","[0.3297959, -0.16444838, 0.9350716, 0.34787956...","[-0.66597974, 0.7140731, 0.43572947, -0.445908...","{'Бренд': ['hoco'], 'Тип': ['Кабель'], 'Цвет т...","[-0.031527344, -0.06875799, 0.03187686, -0.004...","[0.015840497, 0.022141043, -0.03528703, -0.017..."
704096517,Блок питания для ноутбука Asus f5gl (19V 90W 4...,"{'1': 'EPG', '2': 'Электроника', '3': 'Зарядны...",[черный],"[[-0.013610864, -0.68512607, 0.77639246, -1.04...","[0.2785852, -0.16053033, 1.1653559, 1.0619084,...","[-0.7575411, 0.4196694, 0.46428213, -0.4916808...","{'Комплектация': ['Зарядное устройство, сетево...","[-0.023706086, -0.012301952, -0.014316322, -0....","[0.009541714, -0.029578354, -0.015135301, 0.03..."
705874953,Оперативная память HyperX FURY Black DDR4 2666...,"{'1': 'EPG', '2': 'Электроника', '3': 'Операти...",[black],"[[0.34073856, 0.65070343, 0.31146732, 1.261663...","[0.31382418, 0.60041714, 0.3067428, 1.1233345,...","[-0.60506856, 0.4477128, 0.62255704, -0.720129...","{'Тайминги': ['16-18-18-29'], 'Пропускная спос...","[-0.028754005, -0.025122717, 0.048854, -0.0297...","[-0.03909694, 0.040358614, 0.025154725, 0.0568..."


In [8]:
train_df = pd.read_pickle('./dataframes/train_df.pickle')
rtrain_df = pd.read_pickle('./dataframes/rtrain_df.pickle')
test_df = pd.read_pickle('./dataframes/test_df.pickle')
rtest_df = pd.read_pickle('./dataframes/rtest_df.pickle')

In [9]:
cat_features = [
    'cat3', 'cat4',
    'Страна-изготовитель_1', 'Страна-изготовитель_2', 
    'Оперативная память_1', 'Оперативная память_2',
    'Бренд процессора_1', 'Бренд процессора_2',
    'Модуль связи Bluetooth_1', 'Модуль связи Bluetooth_2',
    'Назначение_1', 'Назначение_2',
]
embedding_features = ['main_pic_1', 'main_pic_2', 'name_bert_1', 'name_bert_2', 'name_labse_1', 'name_labse_2']

In [10]:
train_pool = Pool(
    data=train_df.drop(['target']+embedding_features, axis = 1),
    cat_features=cat_features,
)
rtrain_pool = Pool(
    data=rtrain_df.drop(['target']+embedding_features, axis = 1),
    cat_features=cat_features,
)
test_pool = Pool(
    data=test_df.drop(embedding_features, axis = 1),
    cat_features=cat_features,
)
rtest_pool = Pool(
    data=rtest_df.drop(embedding_features, axis = 1),
    cat_features=cat_features,
)

In [11]:
model_cb_pseudo = CatBoostClassifier().load_model('./models/golden_model_cb_pseudo.cbm')

In [12]:
train_pairs['cb_pseudo_pred'] = rtrain_pairs['cb_pseudo_pred'] = \
(model_cb_pseudo.predict_proba(train_pool)[:, 1] + model_cb_pseudo.predict_proba(rtrain_pool)[:, 1]) / 2.

test_pairs['cb_pseudo_pred'] = rtest_pairs['cb_pseudo_pred'] = \
(model_cb_pseudo.predict_proba(test_pool)[:, 1] + model_cb_pseudo.predict_proba(rtest_pool)[:, 1]) / 2.

In [13]:
train_pool = Pool(
    data=train_df.drop(['target'], axis = 1),
    cat_features=cat_features,
    embedding_features=embedding_features
)
rtrain_pool = Pool(
    data=rtrain_df.drop(['target'], axis = 1),
    cat_features=cat_features,
    embedding_features=embedding_features
)
test_pool = Pool(
    data=test_df,
    cat_features=cat_features,
    embedding_features=embedding_features
)
rtest_pool = Pool(
    data=rtest_df,
    cat_features=cat_features,
    embedding_features=embedding_features
)

In [14]:
model_cb = CatBoostClassifier().load_model('./models/golden_model_cb.cbm')

In [15]:
train_pairs['cb_pred'] = rtrain_pairs['cb_pred'] = \
(model_cb.predict_proba(train_pool)[:, 1] + model_cb.predict_proba(rtrain_pool)[:, 1]) / 2.

test_pairs['cb_pred'] = rtest_pairs['cb_pred'] = \
(model_cb.predict_proba(test_pool)[:, 1] + model_cb.predict_proba(rtest_pool)[:, 1]) / 2.

In [16]:
val_pairs = train_pairs[pd.read_csv('./datasets/hard_idx.csv', index_col=0).values].copy()
rval_pairs = rtrain_pairs[pd.read_csv('./datasets/hard_idx.csv', index_col=0).values].copy()

In [17]:
mask = pd.read_csv('./datasets/val_idx.csv', index_col=0).values & ~pd.read_csv('./datasets/hard_idx.csv', index_col=0).values
train_pairs = train_pairs[mask].copy()
rtrain_pairs = rtrain_pairs[mask].copy()

In [18]:
train_pairs

,target,variantid1,variantid2,cb_pseudo_pred,cb_pred
1,1,53062686,536165289,0.180415,0.175911
6,1,62485795,62487150,0.293636,0.396222
7,1,64634116,539055461,0.632096,0.656641
8,1,67137384,781488410,0.301474,0.385981
10,1,70799285,72938934,0.586648,0.562208
...,...,...,...,...,...
306502,0,795842312,795842564,0.032249,0.022872
306510,0,801461635,801461672,0.051367,0.044114
306520,0,810680230,820119986,0.062628,0.050127
306524,0,814169571,822101626,0.085002,0.093381


In [19]:
val_pairs

,target,variantid1,variantid2,cb_pseudo_pred,cb_pred
16,0,78528663,551861879,0.031700,0.025394
48,1,89571302,590187339,0.879985,0.892545
65,1,90672972,355898383,0.985500,0.980606
66,1,90730073,736643943,0.845335,0.894928
77,1,91260321,422212895,0.906132,0.897436
...,...,...,...,...,...
306485,1,788619018,788623137,0.837735,0.851541
306492,0,794450037,816083556,0.047836,0.056798
306503,0,796354669,796354893,0.024209,0.026371
306511,0,801747460,801760537,0.022295,0.022546


In [20]:
test_pairs

,variantid1,variantid2,cb_pseudo_pred,cb_pred
0,52076340,290590137,0.085118,0.072876
1,64525522,204128919,0.065193,0.099754
2,77243372,479860557,0.321192,0.289299
3,86065820,540678372,0.521682,0.431156
4,91566575,258840506,0.221161,0.324661
...,...,...,...,...
18079,666998614,667074522,0.031549,0.030652
18080,670036240,670048449,0.284579,0.327252
18081,670284509,684323809,0.575465,0.632305
18082,692172005,704805270,0.828207,0.858653


# Features

In [21]:
def calc_dists(df, prefix, embs_1, embs_2):
    l1_dists, l2_dists, cos_dists = [], [], []
    for emb_1, emb_2 in zip(embs_1, embs_2):
        len_1 = (emb_1**2).sum()**0.5
        len_2 = (emb_2**2).sum()**0.5
        l1_dists.append(
            np.abs(emb_1 - emb_2).sum()
        )
        l2_dists.append(
            ((emb_1 - emb_2)**2).sum()**0.5
        )
        cos_dists.append(
            (emb_1 @ emb_2) / len_1 / len_2
        )
    df[f'{prefix}_l1_dist'] = l1_dists
    df[f'{prefix}_l2_dist'] = l2_dists
    df[f'{prefix}_cos_dist'] = cos_dists

In [53]:
def make_features(pairs, data):
    gc.collect()
    df = pairs.copy()
    neuro_1 = data.loc[df.variantid1, 'neuro_768'].values
    neuro_2 = data.loc[df.variantid2, 'neuro_768'].values
    calc_dists(
        df, 'neuro', 
        neuro_1,
        neuro_2
    )
    df[[f'neuro_emb{i}_1' for i in range(768)]] = list(neuro_1)
    df[[f'neuro_emb{i}_2' for i in range(768)]] = list(neuro_2)
    df = df.copy()
    return df.drop(['variantid1', 'variantid2'], axis=1)

In [54]:
%%time
train_df = make_features(train_pairs, train_data)
train_df

CPU times: total: 1min 6s
Wall time: 1min 6s


,target,cb_pseudo_pred,cb_pred,neuro_l1_dist,neuro_l2_dist,neuro_cos_dist,neuro_emb0_1,neuro_emb1_1,neuro_emb2_1,neuro_emb3_1,neuro_emb4_1,neuro_emb5_1,neuro_emb6_1,neuro_emb7_1,neuro_emb8_1,neuro_emb9_1,neuro_emb10_1,neuro_emb11_1,neuro_emb12_1,neuro_emb13_1,neuro_emb14_1,neuro_emb15_1,neuro_emb16_1,neuro_emb17_1,neuro_emb18_1,neuro_emb19_1,neuro_emb20_1,neuro_emb21_1,neuro_emb22_1,neuro_emb23_1,neuro_emb24_1,neuro_emb25_1,neuro_emb26_1,neuro_emb27_1,neuro_emb28_1,neuro_emb29_1,neuro_emb30_1,neuro_emb31_1,neuro_emb32_1,neuro_emb33_1,neuro_emb34_1,neuro_emb35_1,neuro_emb36_1,neuro_emb37_1,neuro_emb38_1,neuro_emb39_1,neuro_emb40_1,neuro_emb41_1,neuro_emb42_1,neuro_emb43_1,neuro_emb44_1,neuro_emb45_1,neuro_emb46_1,neuro_emb47_1,neuro_emb48_1,neuro_emb49_1,neuro_emb50_1,neuro_emb51_1,neuro_emb52_1,neuro_emb53_1,neuro_emb54_1,neuro_emb55_1,neuro_emb56_1,neuro_emb57_1,neuro_emb58_1,neuro_emb59_1,neuro_emb60_1,neuro_emb61_1,neuro_emb62_1,neuro_emb63_1,neuro_emb64_1,neuro_emb65_1,neuro_emb66_1,neuro_emb67_1,neuro_emb68_1,neuro_emb69_1,neuro_emb70_1,neuro_emb71_1,neuro_emb72_1,neuro_emb73_1,neuro_emb74_1,neuro_emb75_1,neuro_emb76_1,neuro_emb77_1,neuro_emb78_1,neuro_emb79_1,neuro_emb80_1,neuro_emb81_1,neuro_emb82_1,neuro_emb83_1,neuro_emb84_1,neuro_emb85_1,neuro_emb86_1,neuro_emb87_1,neuro_emb88_1,neuro_emb89_1,neuro_emb90_1,neuro_emb91_1,neuro_emb92_1,neuro_emb93_1,neuro_emb94_1,neuro_emb95_1,neuro_emb96_1,neuro_emb97_1,neuro_emb98_1,neuro_emb99_1,neuro_emb100_1,neuro_emb101_1,neuro_emb102_1,neuro_emb103_1,neuro_emb104_1,neuro_emb105_1,neuro_emb106_1,neuro_emb107_1,neuro_emb108_1,neuro_emb109_1,neuro_emb110_1,neuro_emb111_1,neuro_emb112_1,neuro_emb113_1,neuro_emb114_1,neuro_emb115_1,neuro_emb116_1,neuro_emb117_1,neuro_emb118_1,neuro_emb119_1,neuro_emb120_1,neuro_emb121_1,...,neuro_emb640_2,neuro_emb641_2,neuro_emb642_2,neuro_emb643_2,neuro_emb644_2,neuro_emb645_2,neuro_emb646_2,neuro_emb647_2,neuro_emb648_2,neuro_emb649_2,neuro_emb650_2,neuro_emb651_2,neuro_emb652_2,neuro_emb653_2,neuro_emb654_2,neuro_emb655_2,neuro_emb656_2,neuro_emb657_2,neuro_emb658_2,neuro_emb659_2,neuro_emb660_2,neuro_emb661_2,neuro_emb662_2,neuro_emb663_2,neuro_emb664_2,neuro_emb665_2,neuro_emb666_2,neuro_emb667_2,neuro_emb668_2,neuro_emb669_2,neuro_emb670_2,neuro_emb671_2,neuro_emb672_2,neuro_emb673_2,neuro_emb674_2,neuro_emb675_2,neuro_emb676_2,neuro_emb677_2,neuro_emb678_2,neuro_emb679_2,neuro_emb680_2,neuro_emb681_2,neuro_emb682_2,neuro_emb683_2,neuro_emb684_2,neuro_emb685_2,neuro_emb686_2,neuro_emb687_2,neuro_emb688_2,neuro_emb689_2,neuro_emb690_2,neuro_emb691_2,neuro_emb692_2,neuro_emb693_2,neuro_emb694_2,neuro_emb695_2,neuro_emb696_2,neuro_emb697_2,neuro_emb698_2,neuro_emb699_2,neuro_emb700_2,neuro_emb701_2,neuro_emb702_2,neuro_emb703_2,neuro_emb704_2,neuro_emb705_2,neuro_emb706_2,neuro_emb707_2,neuro_emb708_2,neuro_emb709_2,neuro_emb710_2,neuro_emb711_2,neuro_emb712_2,neuro_emb713_2,neuro_emb714_2,neuro_emb715_2,neuro_emb716_2,neuro_emb717_2,neuro_emb718_2,neuro_emb719_2,neuro_emb720_2,neuro_emb721_2,neuro_emb722_2,neuro_emb723_2,neuro_emb724_2,neuro_emb725_2,neuro_emb726_2,neuro_emb727_2,neuro_emb728_2,neuro_emb729_2,neuro_emb730_2,neuro_emb731_2,neuro_emb732_2,neuro_emb733_2,neuro_emb734_2,neuro_emb735_2,neuro_emb736_2,neuro_emb737_2,neuro_emb738_2,neuro_emb739_2,neuro_emb740_2,neuro_emb741_2,neuro_emb742_2,neuro_emb743_2,neuro_emb744_2,neuro_emb745_2,neuro_emb746_2,neuro_emb747_2,neuro_emb748_2,neuro_emb749_2,neuro_emb750_2,neuro_emb751_2,neuro_emb752_2,neuro_emb753_2,neuro_emb754_2,neuro_emb755_2,neuro_emb756_2,neuro_emb757_2,neuro_emb758_2,neuro_emb759_2,neuro_emb760_2,neuro_emb761_2,neuro_emb762_2,neuro_emb763_2,neuro_emb764_2,neuro_emb765_2,neuro_emb766_2,neuro_emb767_2
1,1,0.180415,0.175911,7.836064,0.349518,0.938919,-0.052533,-0.010321,-0.018409,0.019210,-0.035565,-0.021615,-0.007733,0.009223,-0.018848,-0.005369,0.048870,-0.036869,0.024449,-0.073497,0.051336,0.019532,0.027405,-0.020072,-0.051782,0.005662,0.033468,0.012370,-0.021671,0.032232,0.08

In [55]:
rtrain_df = make_features(rtrain_pairs, train_data)
val_df = make_features(val_pairs, train_data)
rval_df = make_features(rval_pairs, train_data)

# Model

In [56]:
all_train = pd.concat([train_df, rtrain_df], axis=0)
all_val = pd.concat([val_df, rval_df], axis=0)
all_train

,target,cb_pseudo_pred,cb_pred,neuro_l1_dist,neuro_l2_dist,neuro_cos_dist,neuro_emb0_1,neuro_emb1_1,neuro_emb2_1,neuro_emb3_1,neuro_emb4_1,neuro_emb5_1,neuro_emb6_1,neuro_emb7_1,neuro_emb8_1,neuro_emb9_1,neuro_emb10_1,neuro_emb11_1,neuro_emb12_1,neuro_emb13_1,neuro_emb14_1,neuro_emb15_1,neuro_emb16_1,neuro_emb17_1,neuro_emb18_1,neuro_emb19_1,neuro_emb20_1,neuro_emb21_1,neuro_emb22_1,neuro_emb23_1,neuro_emb24_1,neuro_emb25_1,neuro_emb26_1,neuro_emb27_1,neuro_emb28_1,neuro_emb29_1,neuro_emb30_1,neuro_emb31_1,neuro_emb32_1,neuro_emb33_1,neuro_emb34_1,neuro_emb35_1,neuro_emb36_1,neuro_emb37_1,neuro_emb38_1,neuro_emb39_1,neuro_emb40_1,neuro_emb41_1,neuro_emb42_1,neuro_emb43_1,neuro_emb44_1,neuro_emb45_1,neuro_emb46_1,neuro_emb47_1,neuro_emb48_1,neuro_emb49_1,neuro_emb50_1,neuro_emb51_1,neuro_emb52_1,neuro_emb53_1,neuro_emb54_1,neuro_emb55_1,neuro_emb56_1,neuro_emb57_1,neuro_emb58_1,neuro_emb59_1,neuro_emb60_1,neuro_emb61_1,neuro_emb62_1,neuro_emb63_1,neuro_emb64_1,neuro_emb65_1,neuro_emb66_1,neuro_emb67_1,neuro_emb68_1,neuro_emb69_1,neuro_emb70_1,neuro_emb71_1,neuro_emb72_1,neuro_emb73_1,neuro_emb74_1,neuro_emb75_1,neuro_emb76_1,neuro_emb77_1,neuro_emb78_1,neuro_emb79_1,neuro_emb80_1,neuro_emb81_1,neuro_emb82_1,neuro_emb83_1,neuro_emb84_1,neuro_emb85_1,neuro_emb86_1,neuro_emb87_1,neuro_emb88_1,neuro_emb89_1,neuro_emb90_1,neuro_emb91_1,neuro_emb92_1,neuro_emb93_1,neuro_emb94_1,neuro_emb95_1,neuro_emb96_1,neuro_emb97_1,neuro_emb98_1,neuro_emb99_1,neuro_emb100_1,neuro_emb101_1,neuro_emb102_1,neuro_emb103_1,neuro_emb104_1,neuro_emb105_1,neuro_emb106_1,neuro_emb107_1,neuro_emb108_1,neuro_emb109_1,neuro_emb110_1,neuro_emb111_1,neuro_emb112_1,neuro_emb113_1,neuro_emb114_1,neuro_emb115_1,neuro_emb116_1,neuro_emb117_1,neuro_emb118_1,neuro_emb119_1,neuro_emb120_1,neuro_emb121_1,...,neuro_emb640_2,neuro_emb641_2,neuro_emb642_2,neuro_emb643_2,neuro_emb644_2,neuro_emb645_2,neuro_emb646_2,neuro_emb647_2,neuro_emb648_2,neuro_emb649_2,neuro_emb650_2,neuro_emb651_2,neuro_emb652_2,neuro_emb653_2,neuro_emb654_2,neuro_emb655_2,neuro_emb656_2,neuro_emb657_2,neuro_emb658_2,neuro_emb659_2,neuro_emb660_2,neuro_emb661_2,neuro_emb662_2,neuro_emb663_2,neuro_emb664_2,neuro_emb665_2,neuro_emb666_2,neuro_emb667_2,neuro_emb668_2,neuro_emb669_2,neuro_emb670_2,neuro_emb671_2,neuro_emb672_2,neuro_emb673_2,neuro_emb674_2,neuro_emb675_2,neuro_emb676_2,neuro_emb677_2,neuro_emb678_2,neuro_emb679_2,neuro_emb680_2,neuro_emb681_2,neuro_emb682_2,neuro_emb683_2,neuro_emb684_2,neuro_emb685_2,neuro_emb686_2,neuro_emb687_2,neuro_emb688_2,neuro_emb689_2,neuro_emb690_2,neuro_emb691_2,neuro_emb692_2,neuro_emb693_2,neuro_emb694_2,neuro_emb695_2,neuro_emb696_2,neuro_emb697_2,neuro_emb698_2,neuro_emb699_2,neuro_emb700_2,neuro_emb701_2,neuro_emb702_2,neuro_emb703_2,neuro_emb704_2,neuro_emb705_2,neuro_emb706_2,neuro_emb707_2,neuro_emb708_2,neuro_emb709_2,neuro_emb710_2,neuro_emb711_2,neuro_emb712_2,neuro_emb713_2,neuro_emb714_2,neuro_emb715_2,neuro_emb716_2,neuro_emb717_2,neuro_emb718_2,neuro_emb719_2,neuro_emb720_2,neuro_emb721_2,neuro_emb722_2,neuro_emb723_2,neuro_emb724_2,neuro_emb725_2,neuro_emb726_2,neuro_emb727_2,neuro_emb728_2,neuro_emb729_2,neuro_emb730_2,neuro_emb731_2,neuro_emb732_2,neuro_emb733_2,neuro_emb734_2,neuro_emb735_2,neuro_emb736_2,neuro_emb737_2,neuro_emb738_2,neuro_emb739_2,neuro_emb740_2,neuro_emb741_2,neuro_emb742_2,neuro_emb743_2,neuro_emb744_2,neuro_emb745_2,neuro_emb746_2,neuro_emb747_2,neuro_emb748_2,neuro_emb749_2,neuro_emb750_2,neuro_emb751_2,neuro_emb752_2,neuro_emb753_2,neuro_emb754_2,neuro_emb755_2,neuro_emb756_2,neuro_emb757_2,neuro_emb758_2,neuro_emb759_2,neuro_emb760_2,neuro_emb761_2,neuro_emb762_2,neuro_emb763_2,neuro_emb764_2,neuro_emb765_2,neuro_emb766_2,neuro_emb767_2
1,1,0.180415,0.175911,7.836064,0.349518,0.938919,-0.052533,-0.010321,-0.018409,0.019210,-0.035565,-0.021615,-0.007733,0.009223,-0.018848,-0.005369,0.048870,-0.036869,0.024449,-0.073497,0.051336,0.019532,0.027405,-0.020072,-0.051782,0.005662,0.033468,0.012370,-0.021671,0.032232,0.08

In [57]:
cat_features = []
embedding_features = []

In [58]:
train_pool = Pool(
    data=all_train.drop('target', axis = 1),
    label=all_train['target'].values,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=all_train['cb_pseudo_pred']
)

val_pool = Pool(
    data=all_val.drop('target', axis = 1),
    label=all_val['target'].values,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=all_val['cb_pseudo_pred']
)

In [77]:
params = {
    'loss_function': 'CrossEntropy',
    'eval_metric': 'PRAUC',
    'task_type': 'CPU',
    'max_depth': 5,
    'learning_rate': 0.05
}

In [78]:
%%time
model_cb = CatBoostClassifier(**params, random_seed=56, cat_features=cat_features, embedding_features=embedding_features)
model_cb.fit(train_pool, eval_set=val_pool, verbose=250, plot=True, use_best_model=True, early_stopping_rounds=300)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9064039	test: 0.8999562	best: 0.8999562 (0)	total: 242ms	remaining: 4m 1s
250:	learn: 0.9121093	test: 0.9034555	best: 0.9034676 (232)	total: 26.2s	remaining: 1m 18s
500:	learn: 0.9162647	test: 0.9039775	best: 0.9039800 (484)	total: 51.4s	remaining: 51.1s
750:	learn: 0.9197125	test: 0.9040148	best: 0.9040510 (618)	total: 1m 16s	remaining: 25.4s
999:	learn: 0.9228259	test: 0.9040280	best: 0.9041264 (898)	total: 1m 41s	remaining: 0us

bestTest = 0.904126355
bestIteration = 898

Shrink model to first 899 iterations.
CPU times: total: 51min 46s
Wall time: 1min 43s


In [79]:
np.max(model_cb.get_evals_result()['validation']['PRAUC'])

0.9041263550318052

In [70]:
model_cb.get_feature_importance(prettified=True)

,Feature Id,Importances
0,cb_pred,25.682825
1,cb_pseudo_pred,19.574883
2,neuro_l2_dist,3.803229
3,neuro_cos_dist,3.585549
4,neuro_l1_dist,2.026479
...,...,...
1536,neuro_emb747_2,0.000000
1537,neuro_emb748_2,0.000000
1538,neuro_emb754_2,0.000000
1539,neuro_emb758_2,0.000000


# Inference

In [47]:
test_df = make_features(test_pairs, test_data)
rtest_df = make_features(rtest_pairs, test_data)

In [48]:
test_pool = Pool(
    data=test_df,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=test_df['cb_pseudo_pred']
)

rtest_pool = Pool(
    data=rtest_df,
    cat_features=cat_features,
    embedding_features=embedding_features,
    baseline=rtest_df['cb_pseudo_pred']
)

In [49]:
preds = model_cb.predict_proba(test_pool)[:,1]
preds

array([0.06408981, 0.0779099 , 0.18736944, ..., 0.43052199, 0.87717535,
       0.45431952])

In [50]:
rpreds = model_cb.predict_proba(rtest_pool)[:,1]
rpreds

array([0.06408981, 0.0779099 , 0.18736944, ..., 0.43052199, 0.87717535,
       0.45431952])

In [51]:
final_preds = (preds + rpreds) / 2.
final_preds

array([0.06408981, 0.0779099 , 0.18736944, ..., 0.43052199, 0.87717535,
       0.45431952])

In [52]:
test_pairs['target'] = final_preds
test_pairs.to_csv('./golden_submit_ens.csv', index=False)
test_pairs.drop('target', axis=1, inplace=True)